- 사전에 정의된 LLM 프롬프트 템플릿은 모델에게 전달할 지시 사항, 컨텍스트, 제약 조건 등을 담고 있다
- 변수를 포함하여 다양한 입력 값에 대해 일관된 출력을 생성
- 언어 모델은 사용자 질문에 대해 최적화된 방식으로 응답을 생성하여, 시스템 전체의 성능과 사용자 만족도에 직접적인 영향

## 01 프롬프트 템플릿 만들기

- **프롬프트 템플릿**의 구조
    - 지시사항(instruction)
    - 사용자가 입력한 질문(question)
    - 검색된 정보인 문맥(context)
- 프롬프트 템플릿의 예시

```
당신은 질문-답변(Question-Answer) Task를 수행하는 AI 어시스턴트입니다.
검색된 문맥(context)을 사용하여 질문(question)에 답변하세요.
만약, 문맥(context)에 답변이 없다면 "모른다"라고 답변하세요.

한국어로 대답하세요.

#Question:
{이곳에 사용자가 입력한 질문이 들어갑니다}

#Context:
{이곳에 검색된 정보가 들어갑니다}
```

- 프롬프트 템플릿 만드는 방법
  - from_template() 메서드 사용하여 PromptTemplate 객체 생성
  - PromptTemplate 객체 생성과 동시에 프롬프트 생성


In [1]:
from dotenv import load_dotenv
from langchain_teddynote import logging

load_dotenv()
logging.langsmith("CH02-Prompt")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH02-Prompt


In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

- 템플릿 문자열 안에 변수를 넣고, 나중에 실행할 때 필요한 값을 채워 넣어 완성된 프롬프트를 LLM에 전달하는 구조

In [3]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate

template = "{country}의 수도는 어디인가요?"

prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

- 작업 중간에 결과를 확인하고 싶다면 format() 메서드로 변수에 값을 넣어 볼 수 있다

In [4]:
prompt = prompt.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

- 프롬프트 템플릿과 LLM을 체인으로 연결

In [5]:
template = "{country}의 수도는 어디인가요?"
prompt = PromptTemplate.from_template(template)
chain = prompt | llm
chain.invoke("대한민국").content

'대한민국의 수도는 서울 입니다.'

- 템플릿 문자열을 정의한 후 PromptTemplate 객체를 생성할 때 template과 input_variables를 직접 지정

In [6]:
template = "{country}의 수도는 어디인가요?"

prompt = PromptTemplate(
    template=template,
    input_variables=["country"],)

prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

country 변수를 채워 넣어 프롬프트 생성

In [7]:
prompt.format(country="대한민국")

'대한민국의 수도는 어디인가요?'

partial_variables는 보통 두 개 이상의 변수를 처리하는 경우 일부 변수를 미리 확정 지을 필요가 있을 때 유용

In [8]:
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={"country2": "미국"},
)

prompt

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '미국'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [9]:
prompt.format(country1="대한민국")

'대한민국과 미국의 수도는 각각 어디인가요?'

- partial() 메서드를 사용하여 일부 변수를 미리 지정 가능

In [10]:
prompt_partial = prompt.partial(country2="캐나다")
prompt_partial

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '캐나다'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [11]:
prompt_partial.format(country1="대한민국")

'대한민국과 캐나다의 수도는 각각 어디인가요?'

- partial()로 미리 country2에 '캐나다' 값을 채워둔 템플릿을 LLM과 연결하여 체인 생성
- 다음 남은 변수인 country1에 값을 입력해 invoke() 메서드 실행

In [12]:
chain = prompt_partial | llm
chain.invoke({"country1": "대한민국"}).content

'대한민국의 수도는 서울이며, 캐나다의 수도는 오타와입니다.'

- partial()로 설정된 값도 새로운 값으로 대체 가능

In [13]:
chain.invoke({"country1": "대한민국", "country2": "호주"}).content

'대한민국의 수도는 서울이며, 호주의 수도는 캔버라입니다.'

## 02 부분변수 활용하기

- **부분 변수**는 주로 함수를 사용하여 프롬프트의 일부를 미리 설정하는 것

In [14]:
from datetime import datetime
datetime.now().strftime("%B %d")

'February 04'

In [15]:
def get_today():
    return datetime.now().strftime("%B %d")

In [16]:
prompt = PromptTemplate(
    template="오늘은 {today}입니다. 오늘이 생일인 유명인 {n}명을 나열해주세요. 생년월일을 표기해주세요",
    input_variables=["n"],
    partial_variables={"today": get_today()},
)
prompt.format(n=3)

'오늘은 February 04입니다. 오늘이 생일인 유명인 3명을 나열해주세요. 생년월일을 표기해주세요'

In [17]:
chain = prompt | llm
print(chain.invoke(3).content)

1. 로사 파크스 (Rosamund Mary Ellen Pike) - 1979년 2월 27일
2. 남궁민 (Namkoong Min) - 1978년 2월 12일
3. 조심 (Jim Jefferies) - 1977년 2월 6일


- 필요에 따라 미리 설정해 둔 변수에 다른 값을 넣어 유연하게 바꿀 수 있다

In [18]:
print(chain.invoke({'today': 'Jan 02', 'n': 3}).content)

1. 김태희 (1980년 3월 9일)
2. 가수 송민호 (민호) (1991년 3월 30일)
3. 배우 이하늬 (1981년 3월 2일)


## 03 YAML 파일로부터 프롬프트 템플릿 로드하기

- 프롬프트가 길어지면 소스 코드 내의 문자열이 너무 길어져서 관리하기 힘들어짐
- yaml 파일에 저장하면 쉽게 관리하고 수정 가능

- 왼쪽에는 키 값, 템플릿에는 실제 프롬프트

```yaml
_type: "prompt"
template: "{fruit}의 색깔이 뭐야?"
input_variables: ["fruit"]
```

In [19]:
from langchain_core.prompts import load_prompt

prompt = load_prompt('../prompts/fruit_color.yaml', encoding="utf-8")
prompt

PromptTemplate(input_variables=['fruit'], input_types={}, partial_variables={}, template='{fruit}의 색깔이 뭐야?')

In [20]:
prompt.format(fruit="사과")

'사과의 색깔이 뭐야?'

- 파이프 연산자(|)를 사용하면 다음줄로 이어서 작성 가능
- 들여쓰기는 두 칸으로 맞춰서 작성

```yaml
_type: "prompt"
template: |
  {country}의 수도에 대해서 알려주세요.
  수도의 특징을 다음의 양식에 맞게 정리해 주세요.
  300자 내외로 작성해 주세요.
  한글로 작성해 주세요.
  ----
  [양식]
  1. 면적
  2. 인구
  3. 역사적 장소
  4. 특산품

  #Answer:
input_variables: ["country"]
```

In [21]:
prompt2 = load_prompt('../prompts/capital.yaml', encoding="utf-8")
print(prompt2.format(country="대한민국"))

대한민국의 수도에 대해서 알려주세요.
수도의 특징을 다음의 양식에 맞게 정리해 주세요.
300자 내외로 작성해 주세요.
한글로 작성해 주세요.
----
[양식]
1. 면적
2. 인구
3. 역사적 장소
4. 특산품

#Answer:



In [22]:
from langchain_core.output_parsers import StrOutputParser
from langchain_teddynote.messages import stream_response

chain = prompt2 | ChatOpenAI(model_name="gpt-4o", temperature=0) | StrOutputParser()
answer = chain.stream({"country": "대한민국"})
stream_response(answer)

1. 면적: 대한민국의 수도인 서울특별시는 약 605.21제곱킬로미터의 면적을 가지고 있습니다. 이는 대한민국 전체 면적의 약 0.6%에 해당합니다.

2. 인구: 서울의 인구는 약 950만 명으로, 대한민국에서 가장 인구가 많은 도시입니다. 다양한 문화와 경제 활동의 중심지로서 많은 사람들이 거주하고 있습니다.

3. 역사적 장소: 서울에는 경복궁, 창덕궁, 덕수궁 등 조선시대의 궁궐들이 있으며, 이외에도 종묘, 남산타워, 한양도성 등 다양한 역사적 명소가 있습니다. 이러한 장소들은 서울의 풍부한 역사와 문화를 보여줍니다.

4. 특산품: 서울은 전통적으로 한복, 한지, 도자기 등이 유명합니다. 또한, 현대에는 다양한 패션, 뷰티 제품과 전자제품 등이 서울을 대표하는 특산품으로 자리 잡고 있습니다.

## 04 ChatPromptTemplate

- 대화형 챗봇과 같이 상호작용하는 시스템에서는 **ChatPromptTemplate**을 사용하면 더 자연스럽고 좋은 답변
- 일반 프롬프트 템플릿은 주로 '이렇게 해줘'처럼 일방적인 지시 사항 하나만 전달
- ChatPromptTemplate은 사용자와 대화를 주고받는 형식으로 프롬프트를 구성하므로 더 유연하고 자연스러운 결과
- 메시지는 튜플 형식으로 구성
- (role, content) 쌍으로 이루어지며, 역할에는 system, human, ai가 있고 메시지를 구성할 때는 반드시 넣어야됨
    - system
        - 시스템 메시지로, 모델에게 역할과 행동 지침을 제공
        - 전역 설정과 관련된 프롬프트이며, 대화 전체에 적용되는 설정
    - user
        - 사용자 메시지로, 사용자가 모델에게 묻거나 요청하는 내용
        - 여러개의 human 메시지를 포함할 수도 있어 대화의 흐름을 더욱 자연스럽게 구성 가능
    - ai
        - AI 메시지로, 모델이 이전에 생성한 응답이나 정보를 포함
        - AI 메시지를 포함하면 대화의 연속성이 유지되어 사용자가 더욱 자연스럽게 AI와 상호작용 가능

- 출력 결과의 messages를 보면 프롬프트 템플릿처럼 단순히 하나의 질문만 담고 있는게 아니라 HumanMessagePromptTemplate 같은 메시지 구조를 담고있어서 대화 메시지를 주고 받을 수 있게 템플릿 정의 가능

In [23]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?")
chat_prompt

ChatPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?'), additional_kwargs={})])

In [24]:
chat_prompt.format(country="대한민국")

'Human: 대한민국의 수도는 어디인가요?'

In [25]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name}입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

- format_messages() 메서드에 필요한 변수 값을 넣으면 템플릿 내 변수가 실제 값으로 치환되어 최종 메시지 생성

In [26]:
messages = chat_template.format_messages(
    name="테디", user_input="당신의 이름은 무엇입니까?"
)
messages

[SystemMessage(content='당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 테디입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='반가워요!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='당신의 이름은 무엇입니까?', additional_kwargs={}, response_metadata={})]

In [28]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke(messages).content

'제 이름은 테디입니다. 필요하신 도움이 있으시면 언제든지 말씀해주세요!'

- LLM과 ChatPromptTemplate을 체인으로 연결하여 사용 가능

In [29]:
chain = chat_template | llm
chain.invoke({"name": "Teddy", "user_input": "당신의 이름은 무엇입니까?"}).content

'제 이름은 테디입니다. 어떻게 도와드릴까요?'

## 05 MessagesPlaceholder

- 대화에서 아직 확정된 메시지는 아니지만 나중에 채워질 메시지를 채우기 위해 임시로 확보한 자리
- 챗봇을 만들 때 대화를 주고받으며 그 내용을 기록하고자 할 때 MessagesPlaceholder를 자주 사용

- AI 어시스턴트의 역할을 정의하고 대화를 요약하는 기능 구현
- conversation이라는 이름으로 대화의 자리를 미리 확보해 두고, 나중에 그 내용을 채울 수 있게 설정해 둔 부분

In [30]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다"),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다"),
    ]
)
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annota

In [36]:
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
        {"role": "human", "content": "안녕하세요! 저는 오늘 새로 입사한 테디입니다. 만나서 반갑습니다."},
        {"role": "ai", "content": "반가워요! 앞으로 잘 부탁드립니다"}
    ],
)
print(formatted_chat_prompt)

System: 당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다
Human: 안녕하세요! 저는 오늘 새로 입사한 테디입니다. 만나서 반갑습니다.
AI: 반가워요! 앞으로 잘 부탁드립니다
Human: 지금까지의 대화를 5 단어로 요약합니다


In [37]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

chain = chat_prompt | llm | StrOutputParser()

- 체인을 실행하면 요약된 메시지 확인 가능

In [44]:
chain.invoke(
    {
        "word_count": 5,
        "conversation": [
            {"role": "human", "content": "안녕하세요! 저는 오늘 새로 입사한 테디입니다. 만나서 반갑습니다."},
            {"role": "ai", "content": "반가워요! 앞으로 잘 부탁드립니다"}
        ],
    }
)

'테디 새 입사, 반가워요!'

## 06 퓨샷 프롬프트

- GPT는 예시를 제공하여 답변 형식을 알려주면 더 나은 결과를 얻을 수 있음
- **퓨삿 기법(FewShotPromptTemplate)**은 복잡한 문맥이나 질문에 대한 답변을 처리할 때 효과적
- 기본적으로 **원샷**과 **퓨샷**을 포함

In [ ]:
- GPT는 예시를 제공하여 답변 형식을 알려주면 더 나은 결과를 얻을 수 있음
- **퓨삿 기법(FewShotPromptTemplate)**은 복잡한 문맥이나 질문에 대한 답변을 처리할 때 효과적